<a href="https://colab.research.google.com/github/priyal6/NLP-Prac/blob/main/pre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

bert mlm

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
import torch.nn.functional as F

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
text = "The capital of France is [MASK]."

In [ ]:
inputs = tokenizer(text, return_tensors = "pt")

In [ ]:
with torch.no_grad():
  outputs = model(**inputs)
  logits = outputs.logits

In [ ]:
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

In [ ]:
mask_token_logits = logits[0, mask_token_index, :]
probs = F.softmax(mask_token_logits, dim=-1)

In [ ]:
top_5 = torch.topk(probs, 5, dim=-1)

for i, (token_id, score) in enumerate(zip(top_5.indices[0], top_5.values[0])):
  predicted_token = tokenizer.decode([token_id])
  print(f"{i+1}. {predicted_token:10s} | Probability: {score.item():.4f}")

1. paris      | Probability: 0.4168
2. lille      | Probability: 0.0714
3. lyon       | Probability: 0.0634
4. marseille  | Probability: 0.0444
5. tours      | Probability: 0.0303


In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

model_name = "textattack/bert-base-uncased-SST-2"

In [ ]:
tokenizer= AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

result = classifier("The movie was a masterpiece. I loved it")

print(result)


Device set to use cpu


[{'label': 'LABEL_1', 'score': 0.9996185302734375}]
